In [1]:
import os
import tensorflow as tf
from keras import Sequential
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

2022-08-30 19:53:18.628528: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-30 19:53:18.631517: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-30 19:53:18.631529: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
len(os.listdir("Chessman-image-dataset/Chess/Bishop"))

87

In [3]:
direk = os.listdir("Chessman-image-dataset/Chess/")

In [4]:
for x in range(len(direk)):
    print(f"Jumlah direktori {direk[x]}" , "=", len(os.listdir(f"Chessman-image-dataset/Chess/{direk[x]}")))

Jumlah direktori Bishop = 87
Jumlah direktori Rook = 102
Jumlah direktori Pawn = 107
Jumlah direktori Knight = 106
Jumlah direktori King = 76
Jumlah direktori Queen = 78


In [5]:
os.listdir(f"Chessman-image-dataset/Chess/{direk[0]}")

['00000158.jpg',
 '00000010.jpg',
 '00000004.jpg',
 '00000011.jpg',
 '00000039.jpg',
 '00000167.jpg',
 '00000007.jpg',
 '00000013.jpg',
 '00000012.jpg',
 '00000006.jpg',
 '00000199.jpg',
 '00000189.jpg',
 '00000002.JPG',
 '00000016.jpg',
 '00000003.png',
 '00000017.jpg',
 '00000188.jpg',
 '00000177.jpg',
 '00000175.jpg',
 '00000149.jpg',
 '00000001.jpg',
 '00000015.png',
 '00000095.gif',
 '00000028.jpg',
 '00000000.JPG',
 '00000174.jpg',
 '00000113.png',
 '00000066.jpg',
 '00000112.jpg',
 '00000138.jpg',
 '00000110.jpg',
 '00000070.jpg',
 '00000071.png',
 '00000115.jpg',
 '00000129.jpg',
 '00000061.jpg',
 '00000075.jpg',
 '00000049.jpg',
 '00000074.jpg',
 '00000114.jpg',
 '00000102.jpg',
 '00000062.jpg',
 '00000089.jpg',
 '00000088.jpg',
 '00000117.jpg',
 '00000103.jpg',
 '00000126.jpg',
 '00000127.jpg',
 '00000079.jpg',
 '00000092.jpg',
 '00000086.png',
 '00000044.png',
 '00000078.jpg',
 '00000130.png',
 '00000120.jpg',
 '00000068.png',
 '00000082.jpg',
 '00000109.jpg',
 '00000121.jpg

# TEKNIK CNN

In [6]:
train_dir = os.path.join('Chessman-image-dataset/Chess')
train_dir

'Chessman-image-dataset/Chess'

# DATA GAMBAR

In [7]:
data_train = ImageDataGenerator(rescale=1./255, 
                             rotation_range=20, 
                             zoom_range=0.2, 
                             shear_range=0.2, 
                             fill_mode = 'nearest', 
                             validation_split=0.1) # set validation split

# PROSES SPLITING GAMBAR

In [8]:
X_train = data_train.flow_from_directory(train_dir, 
                                         target_size=(150, 150), 
                                         batch_size=8, 
                                         class_mode='categorical', 
                                         subset='training') # set as training data

validation_data = data_train.flow_from_directory(train_dir, # same directory as training data
                                        target_size=(150, 150),
                                        batch_size=16,
                                        class_mode='categorical',
                                        subset='validation')

Found 499 images belonging to 6 classes.
Found 52 images belonging to 6 classes.


In [9]:
model = Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),  
    layers.Conv2D(64, (3,3), activation='relu'), 
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),  
    # Flatten the results to feed into a DNN
    layers.Flatten(), 
    # 512 neuron hidden layer
    layers.Dense(512, kernel_regularizer=l2(0.00001), activation='relu'),
    layers.Dense(256, kernel_regularizer=l2(0.00001), activation='relu'),
    layers.Dense(6, activation='softmax')  
])

2022-08-30 19:53:21.993963: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-30 19:53:21.993995: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-30 19:53:21.994016: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (parrot-swiftsf31459): /proc/driver/nvidia/version does not exist
2022-08-30 19:53:21.994274: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
model.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train,
                    validation_data=validation_data,
                    batch_size=300,
                    epochs=30, 
                    verbose=2)

Epoch 1/30
63/63 - 6s - loss: 1.8259 - accuracy: 0.1743 - val_loss: 1.7794 - val_accuracy: 0.1923 - 6s/epoch - 99ms/step
Epoch 2/30
63/63 - 5s - loss: 1.7736 - accuracy: 0.2004 - val_loss: 1.7473 - val_accuracy: 0.3269 - 5s/epoch - 85ms/step
Epoch 3/30
63/63 - 5s - loss: 1.7339 - accuracy: 0.2786 - val_loss: 1.7505 - val_accuracy: 0.2500 - 5s/epoch - 85ms/step
Epoch 4/30
63/63 - 5s - loss: 1.6909 - accuracy: 0.2605 - val_loss: 1.8290 - val_accuracy: 0.2500 - 5s/epoch - 87ms/step
Epoch 5/30
63/63 - 6s - loss: 1.6406 - accuracy: 0.3166 - val_loss: 1.7295 - val_accuracy: 0.2500 - 6s/epoch - 91ms/step
Epoch 6/30
63/63 - 6s - loss: 1.5902 - accuracy: 0.3126 - val_loss: 1.6507 - val_accuracy: 0.3269 - 6s/epoch - 90ms/step
Epoch 7/30
63/63 - 6s - loss: 1.5487 - accuracy: 0.3447 - val_loss: 1.7884 - val_accuracy: 0.2115 - 6s/epoch - 92ms/step
Epoch 8/30
63/63 - 6s - loss: 1.4719 - accuracy: 0.4088 - val_loss: 1.4503 - val_accuracy: 0.3462 - 6s/epoch - 93ms/step
Epoch 9/30
63/63 - 6s - loss: 1.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12,7))
ax1.plot(history.history["accuracy"])
ax1.set_title("ACCURACY")
ax1.set_ylabel("Accuracy")
ax1.set_xlabel("Epoch")

ax2.plot(history.history["val_accuracy"])
ax2.set_title("VALL ACCURACY")
ax2.set_ylabel("Vall Accuracy")
ax2.set_xlabel("Epoch");

ax3.plot(history.history["loss"])
ax3.set_title("LOSS")
ax3.set_ylabel("Loss")
ax3.set_xlabel("Epoch")

ax4.plot(history.history["val_loss"])
ax4.set_title("VALL LOSS")
ax4.set_ylabel("Vall Loss")
ax4.set_xlabel("Epoch");
fig.tight_layout(pad=0.6)